In [ ]:
methane = """5

C\t0.0\t0.0\t0.0
H\t0.5288\t0.1610\t0.9359
H\t0.2051\t0.8240\t-0.6786
H\t0.3345\t-0.9314\t-0.4496
H\t-1.0685\t-0.0537\t0.1921
#     """

geometry = methane

In [ ]:
from nbed.driver import NbedDriver

In [ ]:
n_active_atoms = 2
basis = "STO-3G"
xc_functional = "B3LYP"  # 'lda, vwn' #'B3LYP'
convergence = 1e-10
charge = 0
mu_level_shift = 1e6
run_ccsd_emb = True
run_fci_emb = True
max_ram = 8_000
run_virtual_loc = False

loc_method = "ibo"  # "spade" #"pipekmezey" "ibo" "boys"
projector = "huzinaga"  # "huzinaga","mu" (secret "both" flag for developers)

# note there are a couple of extra developer methods... see code docstrings for further info

experiment = NbedDriver(
    geometry,
    n_active_atoms,
    basis,
    xc_functional,
    projector,
    localization=loc_method,
    convergence=convergence,
    charge=0,
    mu_level_shift=1e6,
    run_ccsd_emb=run_ccsd_emb,
    run_fci_emb=run_fci_emb,
    run_virtual_localization=run_virtual_loc,
    max_ram_memory=max_ram,
    pyscf_print_level=1,
    savefile=None,
    unit="angstrom",
    occupied_threshold=0.95,
    virtual_threshold=0.95,
    max_hf_cycles=50,
    max_dft_cycles=50,
)

access experiment values using
- `experiment._huzinaga` for huzinaga projector exp
- and
- `experiment._mu` for mu projector exp

developer note if `project='both'` then have access to both dictionaries (standard usage will only have 1 to avoid running two expensive calcs)

keys:
- `e_rhf`== HFock-in-DFT embedding 
- `e_ccsd`== CCSD-in-DFT embedding 
- `e_fci`==  FCI-in-DFT embedding 
- `correction`== correction term due to dft active density not being same as density given by active wavefunction used in WF calculation
- `v_emb`==  Matrix of the embedding potential
- `mo_energies_emb_pre_del`== MO energies of all orbitals  
- `mo_energies_emb_post_del`==  MO energies of active orbitals only (environment deleted)
- `scf` == embedded pyscf scf RHF object

In [ ]:
# access experiment huzinaga values

# experiment._huzinaga['e_rhf']
# experiment._huzinaga['e_ccsd']
# experiment._huzinaga['e_fci']

# experiment._huzinaga['correction']
# experiment._huzinaga['v_emb']
# experiment._huzinaga['classical_energy']

# experiment._huzinaga['mo_energies_emb_pre_del']
# experiment._huzinaga['mo_energies_emb_post_del']
# experiment._huzinaga['scf']


{
    "v_emb": experiment._huzinaga["v_emb"],
    "scf": experiment._huzinaga["scf"],
    "mo_energies_emb_pre_del": experiment._huzinaga["mo_energies_emb_pre_del"],
    "mo_energies_emb_post_del": experiment._huzinaga["mo_energies_emb_post_del"],
    "correction": experiment._huzinaga["correction"],
    "e_rhf": experiment._huzinaga["e_rhf"],
    "classical_energy": experiment._huzinaga["classical_energy"],
    "e_ccsd": experiment._huzinaga["e_ccsd"],
    "e_fci": experiment._huzinaga["e_fci"],
}

In [ ]:
## access experiment mu values

# experiment._mu['e_rhf']
# experiment._mu['e_ccsd']
# experiment._mu['e_fci']

# experiment._mu['correction']
# experiment._mu['v_emb']
# experiment._mu['classical_energy']

# experiment._mu['mo_energies_emb_pre_del']
# experiment._mu['mo_energies_emb_post_del']
# experiment._mu['scf']

# {'v_emb'                   :experiment._mu['v_emb'],
#  'scf'                     :experiment._mu['scf'],
#  'mo_energies_emb_pre_del' :experiment._mu['mo_energies_emb_pre_del'],
#  'mo_energies_emb_post_del':experiment._mu['mo_energies_emb_post_del'],
#  'correction'              :experiment._mu['correction'],
#  'e_rhf'                   :experiment._mu['e_rhf'],
#  'classical_energy'        :experiment._mu['classical_energy'],
#  'e_ccsd'                  :experiment._mu['e_ccsd'],
#  'e_fci'                   :experiment._mu['e_fci']}

# Get qubit Hamiltonian

In [ ]:
from nbed.ham_builder import HamiltonianBuilder
from nbed.ham_converter import HamiltonianConverter

In [ ]:
transform = "jordan_wigner"  # fermion to qubit mapping

qham = HamiltonianBuilder(
    scf_method=experiment.embedded_scf,
    constant_e_shift=experiment.classical_energy,
    transform=transform,
).build()

qham

In [ ]:
# convert to different back ends

output = "pennylane"  # "openfermion", "qiskit", "pennylane"


converter = HamiltonianConverter(qham)
hamiltonian = getattr(converter, output.lower())

print(hamiltonian)

In [ ]:
# note convert works by storing Hamiltonian in intermediate dictionary form of
# Paulioperator key and coefficient value

converter._intermediate